In [1]:
import pandas as pd
import numpy as np
import json
import sys
import csv
import os

savedStdout = sys.stdout  # 保存标准输出流

# 显示所有列
pd.set_option('display.max_columns', None)

In [2]:
# 有4个sheet，把这些纵向拼接
data0 = pd.read_excel('data/学生数据  20231010/学生数据  20231010.xlsx',sheet_name=0)
data1 = pd.read_excel('data/学生数据  20231010/学生数据  20231010.xlsx',sheet_name=1)
data2 = pd.read_excel('data/学生数据  20231010/学生数据  20231010.xlsx',sheet_name=2)
data3 = pd.read_excel('data/学生数据  20231010/学生数据  20231010.xlsx',sheet_name=3)

raw_data = pd.concat([data0, data1, data2, data3])
data = raw_data
data

,学校,班级,学生,阅读日期,图书,阅读时长,特别关注,主要影响品格,图书总字数,适读范围,图书内容主题,图书类型,图书年代,作者国别
0,广州市荔湾区流花路小学,六年1班,陈俊瑜,2020-02-20,一起长大的玩具（快乐读书吧·名著阅读课程化丛书·二年级下册）,48,名著类,成长历程,15万字以内,小学低年级,童话寓言,文学故事类,当代,中国
1,广州市荔湾区流花路小学,六年1班,陈俊瑜,2020-02-21,一起长大的玩具（快乐读书吧·名著阅读课程化丛书·二年级下册）,56,名著类,成长历程,15万字以内,小学低年级,童话寓言,文学故事类,当代,中国
2,广州市荔湾区流花路小学,六年1班,陈俊瑜,2020-03-30,漫画中国通史·一,30,中国传统文化类,激励人生,1万字以内,小学低年级,历史通识,图画故事类,当代,中国
3,广州市荔湾区流花路小学,六年1班,陈俊瑜,2020-03-31,漫画中国通史·一,30,中国传统文化类,激励人生,1万字以内,小学低年级,历史通识,图画故事类,当代,中国
4,广州市荔湾区流花路小学,六年1班,陈俊瑜,2020-04-02,漫画中国通史·一,30,中国传统文化类,激励人生,1万字以内,小学低年级,历史通识,图画故事类,当代,中国
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96575,广州市增城区石滩镇麻车小学,六年6班,刘梓萱,2023-01-07,影子猴（台湾儿童文学馆·牧笛奖精品童话）,30,其他,成长历程,1万字以内,小学高年级,童话寓言,文学故事类,当代,中国
96576,广州市增城区石滩镇麻车小学,六年6班,麦雅琳,2023-01-04,哈利波特与魔法石,50,科学幻想类,其他,23万字以内,小学高年级,世界名著,文学故事类,当代,外国
96577,广州市增城区石滩镇麻车小学,六年6班,谢宇杰,2023-01-18,父与子（全集白金版）,30,名著类,成长历程,1万字以内,小学高年级,文学综合,图画故事类,当代,外国
96578,广州市增城区石滩镇麻车小学,六年6班,谢宇杰,2023-01-19,父与子（全集白金版）,30,名著类,成长历程,1万字以内,小学高年级,文学综合,图画故事类,当代,外国


In [3]:
# 清洗 阅读日期 列，看格式是否统一 
data['阅读日期'].info() # output：681959 non-null  datetime64[ns]

<class 'pandas.core.series.Series'>
Int64Index: 681959 entries, 0 to 96579
Series name: 阅读日期
Non-Null Count   Dtype         
--------------   -----         
681959 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 10.4 MB


In [13]:
# 清洗书名
# 创建一个空字典来存放【清洗前书名-清洗后书名】的键值对
book_name_dict = {}
book_name_list = data['图书'].unique().tolist()
for book in book_name_list:
    book_name_dict[book]=book

# 把字典写入 book_name_dict.json 中
with open('../work/output/book_name_dict.json','w') as file:
    json.dump(book_name_dict, file, ensure_ascii=False, indent=4)

In [5]:
book_name_dict

{'一起长大的玩具（快乐读书吧·名著阅读课程化丛书·二年级下册）': '一起长大的玩具（快乐读书吧·名著阅读课程化丛书·二年级下册）',
 '漫画中国通史·一': '漫画中国通史·一',
 '漫画中国通史·三': '漫画中国通史·三',
 '语文教材二年级下册部编版': '语文教材二年级下册部编版',
 '一年级的小蜜瓜': '一年级的小蜜瓜',
 '细菌世界历险记': '细菌世界历险记',
 '安徒生童话（快乐读书吧·名著阅读课程化丛书·三年级上册）': '安徒生童话（快乐读书吧·名著阅读课程化丛书·三年级上册）',
 '格林童话（快乐读书吧·名著阅读课程化丛书·三年级上册）': '格林童话（快乐读书吧·名著阅读课程化丛书·三年级上册）',
 '我今天写什么日记': '我今天写什么日记',
 '世界恐龙大百科': '世界恐龙大百科',
 '变形金刚1（经典双语电影小说）': '变形金刚1（经典双语电影小说）',
 '新奇交通大探秘（酷科学探索纪）': '新奇交通大探秘（酷科学探索纪）',
 '揭秘恐龙（看里面系列·第一辑）': '揭秘恐龙（看里面系列·第一辑）',
 '我的第一套百科全书—交通百科（科普进校园）': '我的第一套百科全书—交通百科（科普进校园）',
 '地铁是怎样建成的': '地铁是怎样建成的',
 '辞旧迎新过大年：春节（中国记忆·传统节日）': '辞旧迎新过大年：春节（中国记忆·传统节日）',
 '高铁出发了（小贝壳绘本馆“中国力量”科学绘本系列）': '高铁出发了（小贝壳绘本馆“中国力量”科学绘本系列）',
 '南极与北极（我的第一本科学漫画书·儿童百问百答13）': '南极与北极（我的第一本科学漫画书·儿童百问百答13）',
 '天气': '天气',
 '我爱洗手，细菌赶走': '我爱洗手，细菌赶走',
 '江城武汉（中国文化地理绘本系列合辑II）': '江城武汉（中国文化地理绘本系列合辑II）',
 '恐龙世界·侏罗纪': '恐龙世界·侏罗纪',
 '汽车（图解交通工具大百科）': '汽车（图解交通工具大百科）',
 '全明星篮球队': '全明星篮球队',
 '中国神话传说（小学语文新课标必读丛书）': '中国神话传说（小学语文新课标必读丛书）',
 '童说岭南——听彭嘉志讲古仔:广饮广食': '童说岭南——听彭嘉志讲古仔:广饮广食',


In [9]:
book_name_dict

{'一起长大的玩具（快乐读书吧·名著阅读课程化丛书·二年级下册）': '一起长大的玩具（快乐读书吧·名著阅读课程化丛书·二年级下册）',
 '漫画中国通史·一': '漫画中国通史·一',
 '漫画中国通史·三': '漫画中国通史·三',
 '语文教材二年级下册部编版': '语文教材二年级下册部编版',
 '一年级的小蜜瓜': '一年级的小蜜瓜',
 '细菌世界历险记': '细菌世界历险记',
 '安徒生童话（快乐读书吧·名著阅读课程化丛书·三年级上册）': '安徒生童话（快乐读书吧·名著阅读课程化丛书·三年级上册）',
 '格林童话（快乐读书吧·名著阅读课程化丛书·三年级上册）': '格林童话（快乐读书吧·名著阅读课程化丛书·三年级上册）',
 '我今天写什么日记': '我今天写什么日记',
 '世界恐龙大百科': '世界恐龙大百科',
 '变形金刚1（经典双语电影小说）': '变形金刚1（经典双语电影小说）',
 '新奇交通大探秘（酷科学探索纪）': '新奇交通大探秘（酷科学探索纪）',
 '揭秘恐龙（看里面系列·第一辑）': '揭秘恐龙（看里面系列·第一辑）',
 '我的第一套百科全书—交通百科（科普进校园）': '我的第一套百科全书—交通百科（科普进校园）',
 '地铁是怎样建成的': '地铁是怎样建成的',
 '辞旧迎新过大年：春节（中国记忆·传统节日）': '辞旧迎新过大年：春节（中国记忆·传统节日）',
 '高铁出发了（小贝壳绘本馆“中国力量”科学绘本系列）': '高铁出发了（小贝壳绘本馆“中国力量”科学绘本系列）',
 '南极与北极（我的第一本科学漫画书·儿童百问百答13）': '南极与北极（我的第一本科学漫画书·儿童百问百答13）',
 '天气': '天气',
 '我爱洗手，细菌赶走': '我爱洗手，细菌赶走',
 '江城武汉（中国文化地理绘本系列合辑II）': '江城武汉（中国文化地理绘本系列合辑II）',
 '恐龙世界·侏罗纪': '恐龙世界·侏罗纪',
 '汽车（图解交通工具大百科）': '汽车（图解交通工具大百科）',
 '全明星篮球队': '全明星篮球队',
 '中国神话传说（小学语文新课标必读丛书）': '中国神话传说（小学语文新课标必读丛书）',
 '童说岭南——听彭嘉志讲古仔:广饮广食': '童说岭南——听彭嘉志讲古仔:广饮广食',
